## 개요
keras tuner는 tensorflow에 대해 최적의 하이퍼 파라미터를 선택하는데 도움을 주는 라이브러리이다. 기계학습에서 적합한 하이퍼파라미터 집합을 선택하는 프로세스를 하이퍼파라미터 튜닝 또는 하이퍼튜닝 이라고 합니다.
1. 히든레이어의 수 및 너비와 같은 모델 선택에 영향을 미치는 모델 하이퍼파라미터
2. SGD의 학습속도 및 KNN 분류기의 최근접 이웃 수와 같은 학습 알고리즘의 속도와 품질에 영향을 미치는 알고리즘 하이퍼파라미터

In [14]:
import tensorflow as tf
from tensorflow import keras
import IPython

In [2]:
!pip install keras-tuner

In [3]:
import keras_tuner as kt

In [4]:
(train_x, train_y), (test_x, test_y) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [8]:
train_x = train_x/255.0
test_x = test_x/255.0

In [12]:
def model_builder(hp):
    model = keras.Sequential()
    model.add(keras.layers.Flatten(input_shape=(28,28)))

    # 첫번째 dense layer의 요소를 튜닝한다. 32~512 사이를 32씩 증가하면 탐색한다.
    hp_units = hp.Int('units', min_value = 32, max_value=512, step=32)
    model.add(keras.layers.Dense(units=hp_units, activation='relu'))
    model.add(keras.layers.Dense(10)) 

    # 옵티마이저의 learning rate를 조정한다.
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer= keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss= keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy']
    )
    return model

### keras tuner 에는 randomsearch, hyperband, bayesianoptimization, sklearn 네가지가 있으며 여기에서는 hyperband를 사용해보겠다.
#### 하이퍼밴드 튜너를 인스턴스화 하려면 최적화할 객체를 선택하고 학습할 최대 에포크 수를 정해야한다.

In [13]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

### 하이퍼파라미터 검색을 실행하기 전에 훈련 단계가 끝날 때마다 훈련 결과를 지우도록 콜백을 정의

In [15]:
class ClearTrainingOutput(keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.daisplay.clear_output(wait = True)

In [17]:
tuner.search(train_x, train_y, epochs = 10, validation_data = (test_x, test_y), callbacks=[ClearTrainingOutput()])

best_hps = tuner.get_best_hyperparameters(num_trials= 1)[0]

print(best_hps.get('units'))
print(best_hps.get('learning_rate'))

Trial 30 Complete [00h 00m 54s]
val_accuracy: 0.8744999766349792

Best val_accuracy So Far: 0.8863000273704529
Total elapsed time: 00h 11m 49s
288
0.001


In [19]:
model = tuner.hypermodel.build(best_hps)
model.fit(train_x, train_y, epochs = 10, validation_data = (test_x, test_y))

Epoch 1/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.4794 - accuracy: 0.8301 - val_loss: 0.4167 - val_accuracy: 0.8495
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3612 - accuracy: 0.8688 - val_loss: 0.3744 - val_accuracy: 0.8662
Epoch 3/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3256 - accuracy: 0.8799 - val_loss: 0.3524 - val_accuracy: 0.8733
Epoch 4/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3011 - accuracy: 0.8889 - val_loss: 0.3509 - val_accuracy: 0.8752
Epoch 5/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2841 - accuracy: 0.8946 - val_loss: 0.3402 - val_accuracy: 0.8807
Epoch 6/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2690 - accuracy: 0.9003 - val_loss: 0.3234 - val_accuracy: 0.8857
Epoch 7/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2551 - accuracy: 0.9046 - val_loss: 0.3337 - val_accuracy:

하이퍼파라미터 검색중에 실행되는 모든 시험에 대한 상세로그와 체크포인트는 my_dir/intro_to_kt에 들어있다. 하이퍼파라미터 검색을 다시 실행하면 로그의 기존상태를 활용하여 검색을 재개한다. 이를 비활성화 하려면 overwrite = True를 사용하면 된다.